# GLM-4.7-Flash RunPod Endpoint Test

테스트 항목:
1. Health Check
2. Model Info
3. Chat Completions
4. Generate Endpoint (SGLang Native)
5. Multi-turn Tool Calling
6. Tokenizer

In [70]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("zai-org/GLM-4.7-Flash", trust_remote_code=True)
print(f"Vocab size: {tokenizer.vocab_size}")
print(f"Model max length: {tokenizer.model_max_length}")

Vocab size: 154820
Model max length: 128000


In [18]:
import os
import requests
import json
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("RUNPOD_API_KEY")
ENDPOINT_ID = os.getenv("ENDPOINT_ID")
BASE_URL = f"https://api.runpod.ai/v2/{ENDPOINT_ID}"

headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}

print(f"Endpoint: {ENDPOINT_ID}")

Endpoint: g4yqdcdwftex3n


## 1. Health Check

In [11]:
response = requests.get(f"{BASE_URL}/health", headers=headers)
print(f"Status: {response.status_code}")
print(response.json())

Status: 200
{'jobs': {'completed': 1, 'failed': 0, 'inProgress': 0, 'inQueue': 0, 'retried': 0}, 'workers': {'idle': 1, 'initializing': 0, 'ready': 1, 'running': 0, 'throttled': 0, 'unhealthy': 0}}


## 2. Model Info

In [ ]:
payload = {"input": {"openai_route": "/v1/models", "openai_input": {}}}

response = requests.post(f"{BASE_URL}/runsync", headers=headers, json=payload)
print(f"Status: {response.status_code}")
print(response.json())

Status: 200
{'delayTime': 1089, 'executionTime': 37, 'id': 'sync-02dc5a10-3f58-44b0-912c-130b7c007ec7-e2', 'output': {'detail': 'Method Not Allowed'}, 'status': 'COMPLETED', 'workerId': '7h5ilcwin1vhrp'}


## 3. Chat Completions

In [ ]:
payload = {
    "input": {
        "openai_route": "/v1/chat/completions",
        "openai_input": {
            "model": "glm-4.7-flash",
            "messages": [
                # {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "안녕! 자기소개 해줘."}
            ],
            "max_tokens": 8192,
            "temperature": 0.7,
        },
    }
}

response = requests.post(f"{BASE_URL}/runsync", headers=headers, json=payload, timeout=300)
print(f"Status: {response.status_code}")
result = response.json()
print(result)

if "output" in result:
    output = result["output"]
    if "choices" in output:
        print("\n--- Thinking ---")
        print(output["choices"][0]["message"]["reasoning_content"])
        if "tool_calls" in output["choices"][0]:
            print("\n--- Tool Calls ---")
            for tool_call in output["choices"][0]["tool_calls"]:
                print(tool_call)
        print("\n\n--- Answer ---")
        print(output["choices"][0]["message"]["content"])

Status: 200
{'delayTime': 1439, 'executionTime': 16305, 'id': 'sync-f78d1435-a35f-4f6f-8927-e77d50e17669-e1', 'output': {'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'matched_stop': 154827, 'message': {'content': '안녕하세요! 반갑습니다.\n\n저는 **지푸AI(Z.ai)**에서 개발한 **GLM**이라는 대규모 언어 모델입니다.\n\n저의 주된 역할은 사용자분께서 원하는 정보를 제공해 드리거나, 글을 작성하고 번역하는 등 다양한 언어 작업을 도와드리는 것입니다. 여러분의 궁금증을 해결해 드리거나 창의적인 아이디어를 냅니다.\n\n오늘 어떤 이야기를 나누거나 어떤 도움을 받고 싶으신가요? 편하게 말씀해 주세요!', 'reasoning_content': '1.  **Analyze the user\'s request:**\n    *   User says: "안녕! 자기소개 해줘." (Hello! Introduce yourself.)\n    *   Intent: The user wants to know who I am, what I do, and perhaps my capabilities.\n    *   Tone: Friendly and casual ("안녕!").\n\n2.  **Identify my core identity:**\n    *   I am a large language model.\n    *   My name is GLM (General Language Model), developed by Z.ai.\n    *   I process and generate human-like text based on the data I was trained on.\n\n3.  **Determine the key information to include

## 4. Generate Endpoint (SGLang Native)

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather information",
            "parameters": {"type": "object", "properties": {"city": {"type": "string"}}, "required": ["city"]},
        },
    }
]
input_ids = tokenizer.apply_chat_template(
    [{"role": "user", "content": "안녕!"}],
    add_generation_prompt=True,
    tools=tools,
    enable_thinking=False,
)["input_ids"]


def generate(input_ids):
    payload = {
        "input": {
            "openai_route": "/generate",
            "openai_input": {
                "input_ids": input_ids,
                "sampling_params": {
                    "max_new_tokens": 512,
                    "temperature": 1.0,
                    "top_p": 0.95,
                },
                "return_logprob": True,
            },
        }
    }

    response = requests.post(f"{BASE_URL}/runsync", headers=headers, json=payload, timeout=300)
    print(f"Status: {response.status_code}")
    result = response.json()
    if "output" in result:
        output = json.loads(result["output"][0])
        print(output["text"])
    print(json.dumps(result, indent=2, ensure_ascii=False))
    return [output_id for logprob, output_id, _ in output["meta_info"]["output_token_logprobs"]]

In [123]:
input_ids += generate(input_ids)

Status: 200
안녕하세요! 반가워요! 😊

오늘 어떤 도움이 필요하신가요? 날씨 정보를 알고 싶으시면 도시 이름을 알려주세요. 그 외에 다른 질문이나 도움이 필요하신 것이 있으시면 편하게 말씀해주세요!
{
  "delayTime": 1763,
  "executionTime": 1292,
  "id": "sync-adf5a016-c459-4a77-8247-d98c6d175440-e1",
  "output": [
    "{\"text\":\"안녕하세요! 반가워요! 😊\\n\\n오늘 어떤 도움이 필요하신가요? 날씨 정보를 알고 싶으시면 도시 이름을 알려주세요. 그 외에 다른 질문이나 도움이 필요하신 것이 있으시면 편하게 말씀해주세요!\",\"output_ids\":[133420,73588,243,90385,0,63303,19931,146444,35533,0,26440,232,271,56881,15115,246,149040,97,135455,37095,146836,150641,16172,81850,19931,35533,30,37001,254,142995,101,59621,18381,139293,34225,28824,114,33347,29217,32146,135455,29217,85325,17851,139293,132919,90901,13,54459,151028,19355,49050,82278,47610,230,51248,12796,59895,135455,37095,146836,150641,16172,81850,143875,35164,33347,29217,32146,140397,116,138017,134594,142995,222,33720,90901,0,154827],\"meta_info\":{\"id\":\"5c10878b241540e08d768b025f2d0ca9\",\"finish_reason\":{\"type\":\"stop\",\"matched\":154827},\"prompt_tokens\":159,\"weight_version\":\"default

In [124]:
tokenizer.decode(input_ids)

'[gMASK]<sop><|system|>\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "get_weather", "description": "Get weather information", "parameters": {"type": "object", "properties": {"city": {"type": "string"}}, "required": ["city"]}}}\n</tools>\n\nFor each function call, output the function name and arguments within the following XML format:\n<tool_call>{function-name}<arg_key>{arg-key-1}</arg_key><arg_value>{arg-value-1}</arg_value><arg_key>{arg-key-2}</arg_key><arg_value>{arg-value-2}</arg_value>...</tool_call><|user|>안녕!<|assistant|></think>안녕하세요! 반가워요! 😊\n\n오늘 어떤 도움이 필요하신가요? 날씨 정보를 알고 싶으시면 도시 이름을 알려주세요. 그 외에 다른 질문이나 도움이 필요하신 것이 있으시면 편하게 말씀해주세요!<|user|>'

## 5. Multi-turn Tool Calling

In [128]:
input_ids += tokenizer.encode("서울이랑 도쿄날씨 알려줄래요?<|assistant|></think>")

In [129]:
input_ids += generate(input_ids)

Status: 200
서울과 도쿄의 날씨 정보를 알아드리겠습니다!<tool_call>get_weather<arg_key>city</arg_key><arg_value>서울</arg_value></tool_call><tool_call>get_weather<arg_key>city</arg_key><arg_value>도쿄</arg_value></tool_call>
{
  "delayTime": 1375,
  "executionTime": 882,
  "id": "sync-22813512-a6ff-4456-acb4-81384cdd2cb1-e1",
  "output": [
    "{\"text\":\"서울과 도쿄의 날씨 정보를 알아드리겠습니다!<tool_call>get_weather<arg_key>city</arg_key><arg_value>서울</arg_value></tool_call><tool_call>get_weather<arg_key>city</arg_key><arg_value>도쿄</arg_value></tool_call>\",\"output_ids\":[26612,139966,53323,135455,168,123,226,20360,37001,254,142995,101,59621,18381,139293,52609,29237,27907,141880,38027,0,154843,455,68852,154847,8923,154848,154849,26612,139966,154850,154844,154843,455,68852,154847,8923,154848,154849,47683,168,123,226,154850,154844,154829],\"meta_info\":{\"id\":\"e52bce4e75524d828955c97433f34d66\",\"finish_reason\":{\"type\":\"stop\",\"matched\":154829},\"prompt_tokens\":258,\"weight_version\":\"default\",\"total_retractions

In [130]:
tokenizer.decode(input_ids)

'[gMASK]<sop><|system|>\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "get_weather", "description": "Get weather information", "parameters": {"type": "object", "properties": {"city": {"type": "string"}}, "required": ["city"]}}}\n</tools>\n\nFor each function call, output the function name and arguments within the following XML format:\n<tool_call>{function-name}<arg_key>{arg-key-1}</arg_key><arg_value>{arg-value-1}</arg_value><arg_key>{arg-key-2}</arg_key><arg_value>{arg-value-2}</arg_value>...</tool_call><|user|>안녕!<|assistant|></think>안녕하세요! 반가워요! 😊\n\n오늘 어떤 도움이 필요하신가요? 날씨 정보를 알고 싶으시면 도시 이름을 알려주세요. 그 외에 다른 질문이나 도움이 필요하신 것이 있으시면 편하게 말씀해주세요!<|user|>서울이랑 도쿄날씨 알려줄래요?<|assistant|></think>서울과 도쿄의 날씨 정보를 알아드리겠습니다!<tool_call>get_weather<arg_key>city</arg_key><arg_value>서울</arg_value></tool_call><tool_call>get_weather<arg_key>city</a

In [131]:
input_ids += tokenizer.encode("<tool_response>{'city': '서울', 'weather': '맑음', 'temperature': '25도'}</tool_response><tool_response>{'city': '도쿄', 'weather': '흐림', 'temperature': '22도'}</tool_response><|assistant|></think>")

In [132]:
input_ids += generate(input_ids)

Status: 200
서울과 도쿄의 날씨를 알려드렸습니다!

🌞 **서울**
- 날씨: 맑음
- 온도: 25도

☁️ **도쿄**
- 날씨: 흐림
- 온도: 22도

서울은 맑은 날씨로 25도의 쾌적한 온도인데 반해, 도쿄는 조금 흐린 날씨입니다. 두 곳 모두 좋은 날씨네요! 다른 도움이 필요하시면 언제든지 말씀해주세요! 😊
{
  "delayTime": 2580,
  "executionTime": 1728,
  "id": "sync-41354e49-90f0-47f7-ad70-5e6c202dfcb7-e1",
  "output": [
    "{\"text\":\"서울과 도쿄의 날씨를 알려드렸습니다!\\n\\n🌞 **서울**\\n- 날씨: 맑음\\n- 온도: 25도\\n\\n☁️ **도쿄**\\n- 날씨: 흐림\\n- 온도: 22도\\n\\n서울은 맑은 날씨로 25도의 쾌적한 온도인데 반해, 도쿄는 조금 흐린 날씨입니다. 두 곳 모두 좋은 날씨네요! 다른 도움이 필요하시면 언제든지 말씀해주세요! 😊\",\"output_ids\":[26612,139966,53323,135455,168,123,226,20360,37001,254,142995,101,18381,139293,132919,29237,21511,116,38027,2219,151598,3070,26612,139966,1019,12,37001,254,142995,101,25,32828,239,48127,198,12,38315,101,47683,25,220,99446,47683,271,154571,30423,3070,47683,168,123,226,1019,12,37001,254,142995,101,25,145500,238,20058,120,198,12,38315,101,47683,25,220,99241,47683,271,26612,139966,33542,32828,239,33542,37001,254,142995,101,17358,220,99446,47683,20360,3315,143975,80311,23508

## 6. Tokenizer Test

In [65]:
# Tokenize test
text = "안녕하세요! GLM-4.7-Flash 테스트입니다."
tokens = tokenizer.encode(text)
print(f"Text: {text}")
print(f"Tokens: {tokens}")
print(f"Token count: {len(tokens)}")
print(f"Decoded: {tokenizer.decode(tokens)}")

Text: 안녕하세요! GLM-4.7-Flash 테스트입니다.
Tokens: [133420, 73588, 243, 90385, 0, 5588, 44, 12, 19, 13, 22, 12, 23809, 10759, 71953, 52836, 78318, 13]
Token count: 18
Decoded: 안녕하세요! GLM-4.7-Flash 테스트입니다.


In [66]:
# Chat template test
messages = [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Hello!"}]

formatted = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print("Chat template:")
print(formatted)

Chat template:
[gMASK]<sop><|system|>You are a helpful assistant.<|user|>Hello!<|assistant|><think>
